# Projet : CryptoBot avec Binance (OPA)

## Phase 2 : Organisation des Données

### Objectif du Document:

Ce doc a pour but de présenter les différentes étapes pour : 
- Réupérer une donnée Historique, 
- Alimenter la Base Mongo, 
- Alimenter la table SQL Histo

### Step 1:  Initialisation des Modules

In [1]:
from Binance.Data import Binance_Histo as Histo
from Binance.Data import Binance_Live as live
from Binance.Dao import Drivers_MongoDB as DAO_MB
from Binance.Dao import Drivers_SQlite as DAO_SQL
from Binance.Utils import Utilitaire as util
from Binance.Utils import Technical_Analyst as util_TA

import pandas as pd

Symbol = 'BTCUSDT'

### Step 2: Chargement Fichiers Historique 

In [2]:
L = list()

X = Histo.Binance_Histo([Symbol], ['15m'], Frequence= 'M', DateDebut = '2020-01-01', DateFin = '2021-12-31')
X.get_ListeFichier()
X.TelechargeFichier()

for Monfichier in X.L_Fichier:
    L.append(Monfichier['Nom'])

print('ci dessous la liste des fichiers à charger')
print(L[0:2])


ci dessous la liste des fichiers à charger
['BTCUSDT-15m-2020-01.csv', 'BTCUSDT-15m-2020-02.csv']


### Step 3: Connexion à la base Mongo DB

In [5]:
# Connexion à la base avec les fichiers à charger
Y = DAO_MB.Drivers_MongoDB(L)
Y.ChargeFichiers()

In [3]:
# Connexion à la base sans fichiers à charger
Y = DAO_MB.Drivers_MongoDB()

In [4]:
# Suppression des collections si besoin
Y.DeleteCollection([Symbol])

In [6]:
print('Liste des Collection :',  Y.get_AllCollection())
print('Liste des Fichiers Chargés de la 1ere collection :', Y.get_ListeFichier(Y.get_AllCollection()[0]))
X.SupprimeFichier()

Liste des Collection : ['BTCUSDT']
Liste des Fichiers Chargés de la 1ere collection : ['BTCUSDT-15m-2020-01.csv', 'BTCUSDT-15m-2020-02.csv', 'BTCUSDT-15m-2020-03.csv', 'BTCUSDT-15m-2020-04.csv', 'BTCUSDT-15m-2020-05.csv', 'BTCUSDT-15m-2020-06.csv', 'BTCUSDT-15m-2020-07.csv', 'BTCUSDT-15m-2020-08.csv', 'BTCUSDT-15m-2020-09.csv', 'BTCUSDT-15m-2020-10.csv', 'BTCUSDT-15m-2020-11.csv', 'BTCUSDT-15m-2020-12.csv', 'BTCUSDT-15m-2021-01.csv', 'BTCUSDT-15m-2021-02.csv', 'BTCUSDT-15m-2021-03.csv', 'BTCUSDT-15m-2021-04.csv', 'BTCUSDT-15m-2021-05.csv', 'BTCUSDT-15m-2021-06.csv', 'BTCUSDT-15m-2021-07.csv', 'BTCUSDT-15m-2021-08.csv', 'BTCUSDT-15m-2021-09.csv', 'BTCUSDT-15m-2021-10.csv', 'BTCUSDT-15m-2021-11.csv', 'BTCUSDT-15m-2021-12.csv']


### Step 4:Connexion à la base SQLite

In [7]:
#Connexion à la base SQLite

PathDatabase = '/home/arnold/ENV_VIRTUEL/ATU_FORMATION/REP_DEV/Projet_OPA/DataBase/SQLite/test.db'
SQL = DAO_SQL.Drivers_SQLite(PathDatabase)


In [8]:
#Reinitialisation de la DataBase (Si besoin)

PathCreateTable = '/home/arnold/ENV_VIRTUEL/ATU_FORMATION/REP_DEV/Projet_OPA/Binance/Dao/Create_DBSQLITE_OPA.sql'
SQL.Re_InitDB(PathCreateTable)

#### Step 4.1: Alimentation Table Dim Temps

In [9]:
#Récupération des données temps depuis la Base Mongo

Liste_Temps = list(Y.DBMongo[Symbol].find({}, {'Detail.Close_time' : 1, 'Detail.Open_time': 1}))

In [10]:
# Formatage du Résultat dns un DataFrame

R = list()
for i in Liste_Temps :
    for y in i['Detail']:
        if y['Open_time'] not in R:
            R.append(y['Open_time'])
        
        if y['Close_time'] not in R:
            R.append(y['Close_time'])
            
DimTemps = pd.DataFrame(R, columns = ['ID_TEMPS'] , dtype='int')

DimTemps.head()

,ID_TEMPS
0,1577837700000
1,1577838599999
2,1577838600000
3,1577839499999
4,1577839500000


In [11]:
# Mise à jour du DataFrame au format de la table DimTemps

DimTemps['SECONDES'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('ss'))
DimTemps['MINUTES'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('mm'))
DimTemps['HEURE'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('HH'))
DimTemps['JOUR'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('DD'))
DimTemps['MOIS'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('MM'))
DimTemps['ANNEE'] = DimTemps['ID_TEMPS'].apply(util.Convertir_Timestamp, formatDate=('YYYY'))
    
DimTemps.head(5)

,ID_TEMPS,SECONDES,MINUTES,HEURE,JOUR,MOIS,ANNEE
0,1577837700000,0,15,1,1,1,2020
1,1577838599999,59,29,1,1,1,2020
2,1577838600000,0,30,1,1,1,2020
3,1577839499999,59,44,1,1,1,2020
4,1577839500000,0,45,1,1,1,2020


In [12]:
# Alimentation Table Dim_Temps

SQL.Alim_DimTemps(DimTemps)
res = SQL.Select('select * from DIM_TEMPS limit 5;')
print(res)


[(1577837700000, '0', '15', '1', '1', '1', '2020', '2023-02-19'), (1577838599999, '59', '29', '1', '1', '1', '2020', '2023-02-19'), (1577838600000, '0', '30', '1', '1', '1', '2020', '2023-02-19'), (1577839499999, '59', '44', '1', '1', '1', '2020', '2023-02-19'), (1577839500000, '0', '45', '1', '1', '1', '2020', '2023-02-19')]


#### Step 4.2: Alimentation Table Dim Symbol

In [13]:
#Récupération des données symbole depuis la Base Mongo

Liste_Symbole = list(Y.DBMongo[Symbol].find({}, {"Symbol" : 1, "Intervalle" : 1, "_id" : 0}))

R2 = list()
for i in Liste_Symbole:
    a = {"NOM_SYMBOL" : i['Symbol'], "INTERVALLE" : i['Intervalle']} 
    if a not in R2:
        R2.append(a)
        
DimSymbol = pd.DataFrame(R2)

In [14]:
# On compléte le df Symbol

DLive = live.Binance_Live()
Info_symbol = DLive.exchange_info(Symbol)


i = {'NOM_SYMBOL' : Info_symbol['symbols'][0]['symbol'],
                              'BaseAsset' : Info_symbol['symbols'][0]['baseAsset'],
                              'QuoteAsset' : Info_symbol['symbols'][0]['quoteAsset']}
df = pd.DataFrame([i ])
DimSymbol = DimSymbol.merge(df, on = 'NOM_SYMBOL' )



In [15]:
SQL.Alim_DimSymbol(DimSymbol)
res = SQL.Select('select * from DIM_SYMBOL limit 5;')
print(res)
            

[(1, 'BTCUSDT', '15m', 'BTC', 'USDT', '2023-02-19')]


#### Step 4.3: Alimentation Table Fait Situation Histo

In [16]:
# Récupération des données Histo de la base Mongo
L = list()

for doc in Y.get_AllDocuments(Symbol):
    for detail in doc['Detail']:
        L.append({'ID_TEMPS' :detail['Close_time'], 
                  'NOM_SYMBOL' : doc['Symbol'], 
                  'INTERVALLE' : doc['Intervalle'], 
                  'VALEUR_COURS' : detail['Close'] ,
                  'IND_QUOTEVOLUME' : detail['Quote_asset_volume'] 
                 })

FaiCoursHisto = pd.DataFrame(L)
FaiCoursHisto['ID_TEMPS'] = FaiCoursHisto['ID_TEMPS'].astype(int)
FaiCoursHisto['IND_QUOTEVOLUME'] = FaiCoursHisto['IND_QUOTEVOLUME'].astype(int)

FaiCoursHisto.head()


,ID_TEMPS,NOM_SYMBOL,INTERVALLE,VALEUR_COURS,IND_QUOTEVOLUME
0,1577838599999,BTCUSDT,15m,7178.45,920702
1,1577839499999,BTCUSDT,15m,7179.56,599479
2,1577840399999,BTCUSDT,15m,7177.02,697429
3,1577841299999,BTCUSDT,15m,7190.86,744089
4,1577842199999,BTCUSDT,15m,7212.10,1811798


In [17]:
#Recherche Id Symbol à partir de la base SQL
L = list()

res = SQL.Select('select ID_SYMBOL,NOM_SYMBOL,INTERVALLE  from DIM_SYMBOL;')

for i in res:
    (a,b,c) = i
    L.append({'ID_SYMBOL' : a, 'NOM_SYMBOL' : b, 'INTERVALLE' : c})
    
df = pd.DataFrame(L)

FaiCoursHisto = FaiCoursHisto.merge(df, how = 'inner')

FaiCoursHisto.head()

,ID_TEMPS,NOM_SYMBOL,INTERVALLE,VALEUR_COURS,IND_QUOTEVOLUME,ID_SYMBOL
0,1577838599999,BTCUSDT,15m,7178.45,920702,1
1,1577839499999,BTCUSDT,15m,7179.56,599479,1
2,1577840399999,BTCUSDT,15m,7177.02,697429,1
3,1577841299999,BTCUSDT,15m,7190.86,744089,1
4,1577842199999,BTCUSDT,15m,7212.10,1811798,1


In [18]:
# Calcul des Indicateurs
FaiCoursHisto['IND_SMA_20'] = util_TA.Calculer_SMA(FaiCoursHisto['VALEUR_COURS'], 20)
FaiCoursHisto['IND_SMA_30'] = util_TA.Calculer_SMA(FaiCoursHisto['VALEUR_COURS'], 30)
FaiCoursHisto['IND_CHANGEPERCENT'] = util_TA.Calculer_Change_Percent(FaiCoursHisto['VALEUR_COURS'])
FaiCoursHisto['IND_STOCH_RSI'] = util_TA.Calculer_RSI_Stochastique(FaiCoursHisto['VALEUR_COURS'])
FaiCoursHisto['IND_RSI'] =  util_TA.Calculer_RSI(FaiCoursHisto['VALEUR_COURS'])
FaiCoursHisto['IND_TRIX'] = util_TA.calculate_trix(FaiCoursHisto['VALEUR_COURS'])

FaiCoursHisto = FaiCoursHisto[['ID_TEMPS',  'ID_SYMBOL','VALEUR_COURS', 'IND_SMA_20', 'IND_SMA_30', 'IND_QUOTEVOLUME', 'IND_CHANGEPERCENT', 'IND_STOCH_RSI', 'IND_RSI', 'IND_TRIX']]

FaiCoursHisto.head(5)

,ID_TEMPS,ID_SYMBOL,VALEUR_COURS,IND_SMA_20,IND_SMA_30,IND_QUOTEVOLUME,IND_CHANGEPERCENT,IND_STOCH_RSI,IND_RSI,IND_TRIX
0,1577838599999,1,7178.45,NaN,NaN,920702,NaN,NaN,NaN,NaN
1,1577839499999,1,7179.56,NaN,NaN,599479,0.015461,NaN,NaN,NaN
2,1577840399999,1,7177.02,NaN,NaN,697429,-0.035391,NaN,NaN,NaN
3,1577841299999,1,7190.86,NaN,NaN,744089,0.192467,NaN,NaN,NaN
4,1577842199999,1,7212.10,NaN,NaN,1811798,0.294505,NaN,NaN,NaN


In [19]:
# Alimentation table SQL

SQL.Alim_FaitSituation_Histo(FaiCoursHisto)
res = SQL.Select('select * from FAIT_SIT_COURS_HIST limit 5;')
print(res)
            

[(1, 1577838599999, 1, 7178.45, None, None, 920702, None, None, None, None, '2023-02-19'), (2, 1577839499999, 1, 7179.56, None, None, 599479, 0.015460557471496609, None, None, None, '2023-02-19'), (3, 1577840399999, 1, 7177.02, None, None, 697429, -0.03539073320124457, None, None, None, '2023-02-19'), (4, 1577841299999, 1, 7190.86, None, None, 744089, 0.19246654781207306, None, None, None, '2023-02-19'), (5, 1577842199999, 1, 7212.1, None, None, 1811798, 0.29450506787205794, None, None, None, '2023-02-19')]
